# Conjoin

> Conjoin is about nested loops.

In [ ]:
from typing import Callable, Collection, Iterator, List, Sequence, Set

## Exercise A

- Write a function `romans() -> Iterator[str]` which yields the roman number from 1 to 4999 in ascending order.

In [ ]:
#collapse
digits0 = ('', 'M', 'MM', 'MMM', 'MMMM')
digits1 = ('', 'C', 'CC', 'CCC', 'CD', 'D', 'DC', 'DCC', 'DCCC', 'CM')
digits2 = ('', 'X', 'XX', 'XXX', 'XL', 'L', 'LX', 'LXX', 'LXXX', 'XC')
digits3 = ('', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX')

def romans1() -> Iterator:
    """
    :return: romans numbers from 1 to 4999
    """
    result = [''] * 4
    for result[0] in digits0:
        for result[1] in digits1:
            for result[2] in digits2:
                for result[3] in digits3:
                    yield ''.join(result)

- Write a function `conjoin(*ts: Iterator) -> Sequence`

In [ ]:
#collapse
def conjoin2(*ts: Iterator) -> Sequence:
    """
    :param gs: list of Iterators
    :return: their cartesian product as list
    """
    values = [None] * len(ts)
    result = []

    def loop(i):
        if i >= len(ts):
            result.append(list(values))
        else:
            for values[i] in ts[i]:
                loop(i + 1)

    loop(0)
    return result

- Write a function `conjoin(*gs: Callable) -> Sequence`

In [ ]:
#collapse
def conjoin1(*gs: Callable) -> Sequence:
    """
    :param gs: list of Callables() -> Iterable
    :return: their cartesian product as list
    """
    values = [None] * len(gs)
    result = []

    def loop(i):
        if i >= len(gs):
            result.append(list(values))
        else:
            for values[i] in gs[i]():
                loop(i + 1)

    loop(0)
    return result

- Write a function `conjoin(*gs: Callable) -> Iterator`

In [ ]:
#collapse
def conjoin(*gs: Callable) -> Iterator:
    """
    :param gs: list of Callables() -> Iterable
    :return: their cartesian product as iterator
    """
    values = [None] * len(gs)

    def loop(i):
        if i >= len(gs):
            yield list(values)
        else:
            for values[i] in gs[i]():
                yield from loop(i + 1)

    yield from loop(0)

- Write a function `romans() -> Iterator[str]` which yields the roman number from 1 to 4999 in ascending order.

In [ ]:
#collapse
d0 = lambda: digits0
d1 = lambda: digits1
d2 = lambda: digits2
d3 = lambda: digits3

def romans() -> Iterator:
    """
    :return: romans numbers from 1 to 4999
    """
    result = conjoin(d0, d1, d2, d3)
    return (''.join(t) for t in result)

## Exercise B

- Write a function `permutations(xs: Collection) -> Iterator` which yields all permutations of `xs`.
  - Don't try this for collections containing more than 12 elements.

In [ ]:
#collapse
def permutations(xs: Collection) -> Iterator:
    """
    :param xs: a collection
    :return: iterator of all permutations of xs
    """
    steps = []
    visited = len(xs) * [None]

    for i in range(len(xs)):
        def step(i: int = i) -> Iterator:
            """
            :param i: level of step
            :return: all objects not previously visited
            """
            for x in xs:
                if x not in visited[:i]:
                    visited[i] = x
                    yield x

        steps.append(step)

    yield from conjoin(*steps)

- Write a function `free_fields(occupied_fields: Sequence, n: int) -> Set` which returns all save positions on row `k` if `len(occupied_fields) = k - 1`

In [ ]:
#collapse
def free_fields(occupied_fields: Sequence, n: int = 8) -> Set:
    """
    :param occupied_fields: occupied_places[i] = field of queen on row i
    :param n: size of board
    :return: set of save fields in line k = len(state)
    Example: occupied_fields = [3, 1] means:
    In row 0 there is a queen on column 3
    In row 1 there is a queen on column 1
    This would return (4, 6, 7) on an 8x8 board
    """
    result = set(range(n))
    k = len(occupied_fields)
    for i in range(k):
        result.discard(occupied_fields[i] - k + i)
        result.discard(occupied_fields[i])
        result.discard(occupied_fields[i] + k - i)
    return result

- Write a function `queens(n: int) -> Iterator` which yields all solutions of the queens problem on a nxn board.

In [ ]:
#collapse
def queens(n: int) -> Iterator:
    """
    :param n: size of board, n >= 0
    :return: Iterator over all solutions of the queens problem
    The for-loop produces an array of generators, rows
    occupied_fields is global for all of them.
    rows[i]() yields all free fields in row i depending on
    occupied fields in rows 0 to i-1
    """
    rows = []
    occupied_fields = n * [None]

    for i in range(n):
        def row(i: int = i) -> Iterator:
            """
            :param i: row to be iterated on
            :return: list of free fields on row i
            """
            for x in free_fields(occupied_fields[:i], n):
                occupied_fields[i] = x
                yield x

        rows.append(row)

    yield from conjoin(*rows)